[员工离职预测训练赛](https://www.dcjingsai.com/common/cmpt/%E5%91%98%E5%B7%A5%E7%A6%BB%E8%81%8C%E9%A2%84%E6%B5%8B%E8%AE%AD%E7%BB%83%E8%B5%9B_%E7%AB%9E%E8%B5%9B%E4%BF%A1%E6%81%AF.html)

In [2]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb
import os

In [3]:
path = os.getcwd()
train = pd.read_csv(os.path.join(path, 'input', 'pfm_train.csv'))
test = pd.read_csv(os.path.join(path, 'input', 'pfm_test.csv'))

In [13]:
train.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EmployeeNumber',
       'EnvironmentSatisfaction', 'Gender', 'JobInvolvement', 'JobLevel',
       'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome',
       'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')

In [38]:
# 标签列
LABEL = "Attrition"
features = list(data.columns)
features.remove(LABEL)

In [16]:
test[LABEL] = -1
data = train.append(test).reset_index(drop=True)

C:\Users\Light\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [18]:
from sklearn.preprocessing import LabelEncoder

In [20]:
for ft in data.select_dtypes("O").columns:
    lbl = LabelEncoder()
    data[ft] = lbl.fit_transform(data[ft].astype(str))

In [24]:
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,2,1,1,4,1,77,1,1,...,3,80,1,7,2,4,7,5,0,7
1,54,0,1,1,1,4,1,1245,4,0,...,1,80,1,33,2,1,5,4,1,4
2,34,1,1,1,7,3,1,147,1,1,...,4,80,0,9,3,3,9,7,0,6
3,39,0,2,1,1,1,1,1026,4,0,...,3,80,1,21,3,3,21,6,11,8
4,28,1,1,1,1,3,3,1111,1,1,...,1,80,2,1,2,3,1,0,0,0


In [58]:
import lightgbm as lgb
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=200, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )
data1 =data[data[LABEL]!=-1][features]
label1 =data[data[LABEL]!=-1][LABEL]
testx= data[data[LABEL]==-1][features]

train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
model.fit(train_x,train_y)
test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration

In [67]:
res = data[data[LABEL]==-1][[LABEL]]
res[LABEL] = model.predict(testx)

In [68]:
res.to_csv(os.path.join(path, 'output', 'submission.csv'), index=False, header=["result"])